# Summary

1. Dimensionality reduction of data through PCA
2. (AR)HMM to quantify dynamics of (dim-reduced) activity

In [1]:
import os
import numpy as np
import scipy.io as sio
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sns.set_style('white')
sns.set_context('talk')

### load data

In [3]:
expt_id = '190424_f3'  # '190424_f3' (run+feed) | '180824_f3r1'
# expt_id = '180824_f3r1'
if expt_id == '190424_f3':
    file_name = 'runAndFeedSample.mat'
elif expt_id == '180824_f3r1':
    file_name = 'runningSample.mat'
file_path = os.path.join('/home/mattw/Dropbox/research-code/', file_name)
mat_contents = sio.loadmat(file_path)
# trialFlag: indexes running/feeding/running components of the experiment
# dOO: ratiometric dF/F for every active cell
# A: spatial footprint of these cells
# legs, stim, and feed: behavioral data

### preprocess data - split into training/testing sets

In [4]:
# Extract the segments of training data
def split_trials(
        n_trials, rng_seed=0, trials_tr=5, trials_val=1, trials_test=1, trials_gap=1):
    """
    Split trials into train/val/test blocks.

    The data is split into blocks that have gap trials between tr/val/test:
    train tr | gap tr | val tr | gap tr | test tr | gap tr

    Args:
        n_trials (int): number of trials to use in the split
        rng_seed (int): numpy random seed for reproducibility
        trials_tr (int): number of train trials per block
        trials_val (int): number of validation trials per block
        trials_test (int): number of test trials per block
        trials_gap (int): number of gap trials between tr/val/test; there will be
            a total of 3 * `trials_gap` gap trials per block

    Returns:
        (dict)
    """

    # same random seed for reproducibility
    np.random.seed(rng_seed)

    tr_per_block = \
        trials_tr + trials_gap + trials_val + trials_gap + trials_test + trials_gap

    n_blocks = int(np.floor(n_trials / tr_per_block))
    leftover_trials = n_trials - tr_per_block * n_blocks
    if leftover_trials > 0:
        offset = np.random.randint(0, high=leftover_trials)
    else:
        offset = 0
    indxs_block = np.random.permutation(n_blocks)

    batch_indxs = {'train': [], 'test': [], 'val': []}
    for block in indxs_block:

        curr_tr = block * tr_per_block + offset
        batch_indxs['train'].append(np.arange(curr_tr, curr_tr + trials_tr))
        curr_tr += (trials_tr + trials_gap)
        batch_indxs['val'].append(np.arange(curr_tr, curr_tr + trials_val))
        curr_tr += (trials_val + trials_gap)
        batch_indxs['test'].append(np.arange(curr_tr, curr_tr + trials_test))

    for dtype in ['train', 'val', 'test']:
        batch_indxs[dtype] = np.concatenate(batch_indxs[dtype], axis=0)

    return batch_indxs

def zscore(data):
    """zscore over axis 0"""
    std = np.std(data, axis=0)
    mean = np.mean(data, axis=0)
    return (np.copy(data) - mean) / std

def cluster(data):
    """reorder axis 1 of a T x N matrix"""
    from sklearn.decomposition import PCA
    from scipy.cluster import hierarchy

    # fit pca to get a feature vector for each neuron
    pca = PCA(n_components=40)
    pca.fit(data_run)
    features = pca.components_.T

    # cluster feature vectors
    # Z = hierarchy.ward(features)
    Z = hierarchy.linkage(
        features, method='single', metric='cosine', optimal_ordering=True)
    leaves = hierarchy.leaves_list(Z)

    # methods:
    # 'single', 'complete', 'average', 'weighted', 'centroid', 'median', 'ward'
    # metrics:
    # ‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘cityblock’, ‘correlation’, ‘cosine’, 
    # ‘dice’, ‘euclidean’, ‘hamming’, ‘jaccard’, ‘jensenshannon’, ‘kulsinski’, 
    # ‘mahalanobis’, ‘matching’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’, 
    # ‘seuclidean’, ‘sokalmichener’, ‘sokalsneath’, ‘sqeuclidean’, ‘yule’

    return data[:, leaves]

def get_lagged_data(data, n_lags=0, lag_type='concat', valid=True):
    """
    data is T x N; for each time point t we concatenate
    the observations at times t-1, ..., t-n_lags

    Args:
        data (T x num_features np array):
        n_lags (int): 0 returns input data
        lag_type (str): 'single' | 'concat'
        valid (bool): only return valid data (ow 0-padding)

    Returns:
        np array: num_dataervations x (num_features * (n_lags + 1))
    """

    data_lagged = [data]

    for lag in range(n_lags):
        data_lag_curr = np.roll(data, shift=lag + 1, axis=0)
        data_lag_curr[0:lag + 1, :] = 0  # zero out first elements
        data_lagged.append(data_lag_curr)

    if lag_type == 'concat':
        data = np.concatenate(data_lagged, axis=1)
    elif lag_type == 'single':
        data = data_lagged[n_lags]
    else:
        raise ValueError('"%s" invalid lag_type')
    
    if valid:
        data = data[n_lags:]
        
    return data


def pca_reduce_lag(data, n_components, n_lags, indxs):
    # TODO!!!
    
    from sklearn.decomposition import PCA
    
    pca = PCA(n_components=n_components)
    data_pca_ = pca.fit_transform(data)

    data_pca = {}
    for dtype in ['train', 'test', 'val']:
        data_segs = []
        for indx in indxs[dtype]:
            data_segs.append(get_lagged_data(
                data_pca_[(indx*trial_len):(indx*trial_len + trial_len)],
                n_lags=n_lags))
        data_pca[dtype] = data_segs

    data_pca['train_all'] = np.concatenate(data_pca['train'], axis=0)
    data_pca['val_all'] = np.concatenate(data_pca['val'], axis=0)
    
    return data_pca_, data_pca


def plot_behavior_predictions(true, pred, slc=(0, 5000)):
    
    fig = plt.figure(figsize=(12, 4))
    plt.plot(true[slice(*slc)], ls='-', color='k', label='True')
    plt.plot(pred[slice(*slc)], ls='--', label='Predicted')
    plt.xlabel('Time')
    plt.yticks([])
    plt.legend(frameon=False)
    plt.show()

    return fig

In [5]:
# get data from matlab dict
run_only = np.squeeze(mat_contents['trialFlag'] == 1)
run_signal = np.squeeze(mat_contents['legs'].T[run_only])
feed_signal = np.squeeze(mat_contents['feed'].T[run_only])
if np.sum(feed_signal) > 0:
    behavior = np.concatenate([run_signal[:, None], feed_signal[:, None]], axis=1)
else:
    behavior = run_signal
data_run = zscore(mat_contents['dOO'].T[run_only])
from scipy import signal
data_run = signal.detrend(data_run, axis=0)
data_run = cluster(data_run)

# split into train/test trials
trial_len = 100  # length of pseudo-trialsl
n_trials = np.floor(data_run.shape[0] / trial_len)
indxs = split_trials(
    n_trials, trials_tr=10, trials_val=2, trials_test=0, trials_gap=0)
data = {}
run = {}
for dtype in ['train', 'test', 'val']:
    data_segs = []
    run_segs = []
    for indx in indxs[dtype]:
        data_segs.append(data_run[(indx*trial_len):(indx*trial_len + trial_len)])
        run_segs.append(run_signal[(indx*trial_len):(indx*trial_len + trial_len)])
    data[dtype] = data_segs
    run[dtype] = run_segs
# for PCA/regression
data['train_all'] = np.concatenate(data['train'], axis=0)
data['val_all'] = np.concatenate(data['val'], axis=0)
run['train_all'] = np.concatenate(run['train'], axis=0)
run['val_all'] = np.concatenate(run['val'], axis=0)

In [15]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import r2_score

# first reduce dimensionality and lag
n_components = 20
n_lags = 0
data_pca_lag_, data_pca_lag = pca_reduce_lag(data_run, n_components, n_lags, indxs)

# chop off run data
run_lag = {}
for dtype in ['train', 'test', 'val']:
    run_segs = []
    for indx in indxs[dtype]:
        run_segs.append(
            run_signal[(indx*trial_len):(indx*trial_len + trial_len)][n_lags:])
    run[dtype] = run_segs
run_lag['train_all'] = np.concatenate(run['train'], axis=0)
run_lag['val_all'] = np.concatenate(run['val'], axis=0)

reg = Ridge(alpha=1e-5, normalize=True)
reg.fit(data_pca_lag['train_all'], run_lag['train_all'])
run_val = reg.predict(data_pca_lag['val_all'])

r2 = r2_score(run_val, run_lag['val_all'])

In [16]:
print(r2)

-0.12436323726823328


In [14]:
data_pca_lag['train_all'].shape

(4850, 80)

In [13]:
data_pca_lag_.shape

(6726, 20)

In [17]:
fig = plot_behavior_predictions(run_lag, reg.predict(data_pca_lag_))

TypeError: unhashable type: 'slice'

<Figure size 864x288 with 0 Axes>

In [ ]:
fig = plot_behavior_predictions(
    run_lag['val_all'], reg.predict(data_pca_lag['val_all']))

In [ ]:
n_lags_list = [0, 1, 2, 3, 4, 6, 8, 10, 12]
r2s = []
for n_lags in n_lags_list:
    # first reduce dimensionality and lag
    n_components = 40
    data_pca_lag_, data_pca_lag = pca_reduce_lag(
        data_run, n_components, n_lags, indxs)
    # chop off run data
    run_lag = {}
    for dtype in ['train', 'test', 'val']:
        run_segs = []
        for indx in indxs[dtype]:
            run_segs.append(
                run_signal[(indx*trial_len):(indx*trial_len + trial_len)][n_lags:])
        run[dtype] = run_segs
    run_lag['train_all'] = np.concatenate(run['train'], axis=0)
    run_lag['val_all'] = np.concatenate(run['val'], axis=0)

    reg = Ridge(alpha=1e-5, normalize=True)
    reg.fit(data_pca_lag['train_all'], run_lag['train_all'])
    run_val = reg.predict(data_pca_lag['val_all'])

    r2s.append(r2_score(run_val, run_lag['val_all']))

In [ ]:
# plot the r2 of the validation data
fig = plt.figure(figsize=(6, 4))
plt.plot(n_lags_list, r2s, ls='-', marker='o')
plt.xlabel('Num lags')
plt.ylabel('$R^2$')
plt.title('Validation data')

In [ ]:
fig = plot_behavior_predictions(
    run_lag['val_all'], reg.predict(data_pca_lag['val_all']))

In [ ]:
coeffs = np.reshape(reg.coef_, (13, 40))

fig = plt.figure(figsize=(6, 4))
plt.imshow(coeffs, cmap='RdBu')
plt.ylabel('Num lags')
plt.xlabel('PCs')
plt.title('Regression weights')

In [ ]:
# project back into neuron space?